## Check List 4 - MotionFilter
* Motion filters are quick decision makers that prevents trying to plan a infeasible motion.
* Here, ReachChecker is tested, compared with actual planning results in random trials
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4-A.3 LatticedChecker**  
  - pick, place
    - single process
    - *multi process is not supported with LatticedChecker, which uses GPU*
    
* You need trained model to use ReachChecker
  - model/latticized/indy7/20210222-134724
  - model/latticized/panda/20210222-145610
  - 백업: 개인 이동식 하드디스크, 강준수
 
* Also, make sure the xacro files for the robot (src/robot/~.xacro) are same as the ones in the model folder

```
LatticedChecker uses tensorflow model on separate python3 program, through a shared memory.
If some error raises with shared memory, try uncomment and run below cell, to reset shared memory.
```

In [1]:
# import SharedArray as sa
# robot_type_name = "indy7"
# sa.delete("shm://{}.grasp_img".format(robot_type_name))
# sa.delete("shm://{}.arm_img".format(robot_type_name))
# sa.delete("shm://{}.rh_mask".format(robot_type_name))
# sa.delete("shm://{}.result".format(robot_type_name))
# sa.delete("shm://{}.query_in".format(robot_type_name))
# sa.delete("shm://{}.response_out".format(robot_type_name))
# sa.delete("shm://{}.query_quit".format(robot_type_name))

## set running directory

In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir

In [4]:
VISUALIZE = True
CLEARANCE = 1e-3

ROBOT_TYPE = RobotType.indy7
ROBOT_NAME = "indy0"
TOOL_XYZ = (0,0,0.14)
TOOL_RPY = (-np.pi/2,0,0)
GRIP_DEPTH = 0.05
HOME_POSE = (0,0,0,0,0,0)

# ROBOT_TYPE = RobotType.panda
# ROBOT_NAME = "panda0"
# TOOL_XYZ = (0,0,0.112)
# TOOL_RPY = (-np.pi/2,0,0)
# GRIP_DEPTH = 0.03
# HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)

## init combined robot config

In [5]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])
TOOL_LINK = crob.get_robot_tip_dict().values()[0]
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

connection command:
indy0: False


## get ghnd with detected robot config

In [6]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


In [7]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [8]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=False, collision=True)

## init planning scene

In [9]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
if ROBOT_TYPE == RobotType.indy7:
    gscene.create_safe(GEOTYPE.MESH, "indy0_gripper_vis", link_name=TOOL_LINK, dims=(0.1,0.1,0.1),
                       center=(0,0,0), rpy=(0,0,np.pi/2), color=(0.1,0.1,0.1,1), display=True, fixed=True, collision=False, 
                       uri="package://my_mesh/meshes/stl/indy_gripper_asm2_res.STL", scale=(1,1,1))

    gscene.create_safe(GEOTYPE.BOX, "indy0_gripper", link_name=TOOL_LINK, dims=(0.06,0.08,0.06),
                       center=(0,0,0.04), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

    gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger1", link_name=TOOL_LINK, dims=(0.03,0.03,0.095),
                       center=(0.006,0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

    gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger2", link_name=TOOL_LINK, dims=(0.03,0.03,0.095),
                       center=(-0.006,0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

    gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger3", link_name=TOOL_LINK, dims=(0.03,0.03,0.095),
                       center=(0.006,-0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

    gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger4", link_name=TOOL_LINK, dims=(0.03,0.03,0.095),
                       center=(-0.006,-0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                   dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## planner

In [12]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
mplan = MoveitPlanner(pscene)
gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
lcheck = LatticedChecker(pscene, gcheck)
checkers_all = [gcheck, rcheck, lcheck]

In [13]:
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

# Object Classes

In [14]:
from pkg.utils.gjk import get_point_list, get_gjk_distance
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

##
# @class ObstacleBase
# @brief base class for obstacle generators
class ObstacleBase:
    RTH_MIN = None ## R: center ~ nearest point
    RTH_MAX = None
    RPY_MIN = None
    RPY_MAX = None
    DIM_MIN = None
    DIM_MAX = None
    GTYPE = None
    COLOR = (0.7,0.7,0.7,1)
    
    def __init__(self, gscene, name, sampler=np.random.uniform, DIM=None, RTH=None, RPY=None):
        self.name = name
        self.DIM = sampler(self.DIM_MIN, self.DIM_MAX) if DIM is None else DIM
        self.RTH = sampler(self.RTH_MIN, self.RTH_MAX) if RTH is None else RTH
        self.RPY = sampler(self.RPY_MIN, self.RPY_MAX) if RPY is None else RPY
        self.RPY[2] += self.RTH[1]
        self.XYZ = np.array(cyl2cart(*self.RTH))
        verts_rotated = np.matmul(Rot_rpy(self.RPY), (DEFAULT_VERT_DICT[self.GTYPE]*self.DIM).transpose())
        xy_normed = self.XYZ[:2]/(np.linalg.norm(self.XYZ[:2])+1e-6)
        verts_r_compo = np.dot(xy_normed, verts_rotated[:2,:])
        self.XYZ[:2] -= xy_normed[:2]*np.min(verts_r_compo)
        self.RTH[0] -= np.min(verts_r_compo)
        self.geometry = gscene.create_safe(gtype=self.GTYPE, name=self.name, link_name="base_link", 
                                  dims=self.DIM, center=tuple(self.XYZ), rpy=self.RPY, 
                                  color=self.COLOR, display=True, collision=True, fixed=True)
        self.subgeo_list = []
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_me, raddii_me = self.geometry.get_vertice_radius()
        verts_me_global = np.add(np.matmul(verts_me, self.geometry.orientation_mat.transpose()), 
                                 self.geometry.center)
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_me_global))-radii-raddii_me < 1e-4
        
##
# @class WorkPlane
# @brief working plane. target and obstacle objects are generated on this plane
class WorkPlane(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.1)
    RTH_MAX = (0.5, +np.pi/2, +0.4)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.4, 0.6, 0.05)
    DIM_MAX = (0.4, 0.6, 0.05)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.8,0.8,0.2,1)
    
    def __init__(self, gscene, name, floor_height=None, *args, **kwargs):
        assert floor_height is not None, "floor_height needed"
        if floor_height > self.RTH_MIN[2]:
            self.RTH_MIN = self.RTH_MIN[:2]+(floor_height,)
        self.H = 0.4
        ObstacleBase.__init__(self, gscene, name, *args, **kwargs)
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_wp = np.multiply(DEFAULT_VERT_DICT[self.GTYPE], tuple(self.DIM[:2])+(self.H,))
        verts_wp_global = np.add(np.matmul(verts_wp, self.geometry.orientation_mat.transpose()), 
                                 np.add(self.geometry.center, (0,0,self.H/2)))
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_wp_global))-radii < 1e-4
            
    
##
# @class Box
# @brief box with the top and the front side open
class Box(WorkPlane):
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    COLOR =  (0.8,0.8,0.2,0.5)
    H_RANGE = (0.3, 0.6)
    THICKNESS = 0.05
    def __init__(self, gscene, name, H=None, **kwargs):
        WorkPlane.__init__(self, gscene=gscene, name=name, **kwargs)
        self.H = np.random.uniform(*self.H_RANGE) if H is None else H

        ## back wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_bw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(self.DIM[0]/2+self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## left wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_lw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, -self.DIM[1]/2-self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## right wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_rw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, self.DIM[1]/2+self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        
##
# @class SideBox
# @brief box with a side face open
class SideBox(Box):
    RTH_MIN = (0.3, -np.pi/2, -0.1)
    RTH_MAX = (0.5, +np.pi/2, +0.4)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.4, 0.6, 0.05)
    DIM_MAX = (0.4, 0.6, 0.05)
    COLOR =  (0.2,0.2,0.8,1)
    H_RANGE = (0.4, 0.4)
    THICKNESS = 0.05
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## top
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_tp", link_name="base_link", 
            dims=(self.DIM[0], self.DIM[1], self.THICKNESS), center=(0, 0, self.H+self.THICKNESS/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

##
# @class TopBox
# @brief box with the top face open
class TopBox(Box):
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    H_RANGE = (0.3, 0.6)
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## front wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_fw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(-self.DIM[0]/2-self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))
        
##
# @class Floor
# @brief Floor - lowerbound of the workspace
class Floor(ObstacleBase):
    RTH_MIN = (0.0, 0, -0.5)
    RTH_MAX = (0.0, 0, -0.5)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    
##
# @class Ceiling
# @brief Ceiling - upperbound of the workspace
class Ceiling(ObstacleBase):
    RTH_MIN = (0.0, 0, 1.0)
    RTH_MAX = (0.0, 0, 2)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.7,0.7,0.7,0.5)
    
##
# @class Wall
# @brief define horizontal boundary of the workspace
class Wall(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (2.0, np.pi, 0)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.1, 6, 3)
    DIM_MAX = (0.1, 6, 3)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.7,0.7,0.7,0.5)
    
##
# @class Pole
# @brief occasional poles that obstruct robot motion
class Pole(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (0.3, +np.pi, 0)
    RPY_MIN = (0, 0, -np.pi/6)
    RPY_MAX = (0, 0, +np.pi/6)
    DIM_MIN = (0.1, 0.1, 4)
    DIM_MAX = (0.1, 0.1, 4)
    GTYPE = GEOTYPE.BOX
    COLOR = (0.7,0.7,0.7,0.3)
    
##
# @class PlaneObstacle
# @brief Obstacles on the workplane
class PlaneObject(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.2)
    RTH_MAX = (0.8, +np.pi/2, +0.5)
    RPY_MIN = (0, 0, -np.pi)
    RPY_MAX = (0, 0, +np.pi)
    DIM_MIN = (0.05, 0.1, 0.2)
    DIM_MAX = (0.05, 0.1, 0.2)
    GTYPE = GEOTYPE.BOX
    COLOR =  (0.2,0.8,0.2,1)
    def __init__(self, gscene, name, workplane, XYZ_LOC=None, **kwargs):
        ObstacleBase.__init__(self, gscene=gscene, name=name, **kwargs)
        verts, radii = self.geometry.get_vertice_radius()
        verts_rot = np.matmul(self.geometry.orientation_mat, verts.transpose()) ## verices with global orientaion
        verts_rot_loc = np.matmul(workplane.geometry.Toff[:3,:3].transpose(), verts_rot) ## verices with local orientaion
        max_verts = np.max(verts_rot_loc, axis=-1)
        min_verts = np.min(verts_rot_loc, axis=-1)
        if XYZ_LOC is None:
            self.XYZ_LOC = np.random.uniform(np.negative(workplane.DIM)/2-min_verts+radii,np.array(workplane.DIM)/2-max_verts-radii)
            self.XYZ_LOC[2] = workplane.DIM[2]/2 + self.DIM[2]/2 + CLEARANCE
        else:
            self.XYZ_LOC = self.XYZ_LOC
        self.XYZ = np.matmul(workplane.geometry.Toff[:3,:3], self.XYZ_LOC) + workplane.geometry.Toff[:3,3]
        self.geometry.set_offset_tf(center = self.XYZ)
        self.RTH = cart2cyl(*self.XYZ)
        gscene.update_marker(self.geometry)
        
        
def clear_class(gscene, key, Nmax):
    for iw in range(Nmax):
        gname = "{}_{}".format(key, iw)
        if gname in gscene.NAME_DICT:
            gscene.remove(gscene.NAME_DICT[gname])

            
def redistribute_class(gscene, obstacle_class, key, Nmax, workplane_avoid=None):
    clear_class(gscene, key, Nmax)
        
    obs_list = []
    for iw in range(np.random.choice(Nmax)):
        obs = obstacle_class(gscene, "{}_{}".format(key, iw))
        while workplane_avoid is not None and workplane_avoid.is_overlapped_with(obs.geometry):
            obs = obstacle_class(gscene, "{}_{}".format(key, iw))
        obs_list.append(obs)
    return obs_list

            
def disperse_objects(gscene, object_class, key, Nmax, workplane_on):
    clear_class(gscene, key, Nmax)
        
    obs_list = []
    while len(obs_list)<Nmax:
        iw = len(obs_list)
        obs = object_class(gscene, "{}_{}".format(key, iw), workplane_on)
        remove_this = False
        for obs_pre in obs_list:
            if obs_pre.is_overlapped_with(obs.geometry):
                remove_this = True
                break
        if remove_this:
            gscene.remove(obs.geometry)
        else:
            obs_list.append(obs)
    return obs_list


def add_object(pscene, obj, HANDLE_THICKNESS=1e-6, HANDLE_COLOR = (1,0,0,0)):
    gscene = pscene.gscene
    handles = []
    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_tp_a", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_tp_b", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bt_a", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,-obj.DIM[2]/2+GRIP_DEPTH/2), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bt_b", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,-obj.DIM[2]/2+GRIP_DEPTH/2), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_ft_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_ft_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bk_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(0,np.pi/2,0), 
                       color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                   parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bk_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    action_points_dict = {"placement": PlacePoint("placement", obj.geometry, [0,0,-obj.DIM[2]/2-CLEARANCE], [0,0,0])}
    action_points_dict.update({handle.name: Grasp2Point(handle.name, handle, None, (0,0,0)) for handle in handles})
    obj_pscene = pscene.create_subject(oname=obj.name, gname=obj.name, _type=CustomObject, 
                                 action_points_dict=action_points_dict)
    return obj_pscene, handles

### Test: pick&place motions

##### test few times with visualization

In [15]:
from pkg.planning.constraint.constraint_common import calc_redundancy
import random

Nmax_obj = 3
VISUALIZE = True
result_dict = defaultdict(list)
N_TRIAL = 5
i_count = 0

while i_count < N_TRIAL:
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    scenario = "pick"
    initial_state = pscene.update_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    
    obj_name, handle_name, actor_name, actor_geo = (
        "obj_0", 
        random.choice(["obj_0_hdl_ft_a", "obj_0_hdl_ft_b", "obj_0_hdl_bk_a", "obj_0_hdl_bk_b"]), 
        "grip0", 
        "gripper")
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=initial_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    for mfilter in [gcheck, rcheck]:
        success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
        if not success_mfilter:
            break
    if not success_mfilter:
        continue
    i_count += 1
        
    success_mfilter = lcheck.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values, timeout=10)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        pick_state = pscene.rebind_all(binding_list, LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    obj_name, handle_name, actor_name, actor_geo = "obj_0", "placement", "gp", "gp"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=pick_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    
    for _ in range(10):
        redundancy_dict = {
            obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                      actor_name:{"w":np.random.rand()*2*np.pi}}}
        redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                             (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                            }

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        for mfilter in [gcheck, rcheck]:
            success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
            if not success_mfilter:
                break
        if success_mfilter:
            break
    if not success_mfilter:
        continue
        
    success_mfilter = lcheck.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values, timeout=10)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: True
1/5: Result match for pick: True / True
try transition motion
transition motion tried: True
1/5: Result match for place: True / True
try transition motion
transition motion tried: True
2/5: Result match for pick: True / True
try transition motion
transition motion tried: True
2/5: Result match for place: True / True
try transition motion
transition motion tried: True
3/5: Result match for pick: True / True
try transition motion
transition motion tried: True
3/5: Result match for place: True / True
try transition motion
transition motion tried: True
4/5: Result mismatch for pick: False / True
try transition motion
transition motion tried: False
4/5: Result mismatch for place: True / False
try transition motion
transition motion tried: False
5/5: Result match for pick: False / False


##### test many times without visualization

In [16]:
from pkg.planning.constraint.constraint_common import calc_redundancy
import random

Nmax_obj = 3
VISUALIZE = False
result_dict = defaultdict(list)
N_TRIAL = 50
i_count = 0

while i_count < N_TRIAL:
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    scenario = "pick"
    initial_state = pscene.update_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    
    obj_name, handle_name, actor_name, actor_geo = (
        "obj_0", 
        random.choice(["obj_0_hdl_ft_a", "obj_0_hdl_ft_b", "obj_0_hdl_bk_a", "obj_0_hdl_bk_b"]), 
        "grip0", 
        "gripper")
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=initial_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    for mfilter in [gcheck, rcheck]:
        success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
        if not success_mfilter:
            break
    if not success_mfilter:
        continue
    i_count += 1
        
    success_mfilter = lcheck.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values, timeout=10)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        pick_state = pscene.rebind_all(binding_list, LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    obj_name, handle_name, actor_name, actor_geo = "obj_0", "placement", "gp", "gp"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=pick_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    
    for _ in range(10):
        redundancy_dict = {
            obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                      actor_name:{"w":np.random.rand()*2*np.pi}}}
        redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                             (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                            }

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        for mfilter in [gcheck, rcheck]:
            success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
            if not success_mfilter:
                break
        if success_mfilter:
            break
    if not success_mfilter:
        continue
        
    success_mfilter = lcheck.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values, timeout=10)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: True
1/50: Result mismatch for pick: False / True
try transition motion
transition motion tried: True
1/50: Result match for place: True / True
try transition motion
transition motion tried: True
2/50: Result match for pick: True / True
try transition motion
transition motion tried: True
3/50: Result match for pick: True / True
try transition motion
transition motion tried: True
3/50: Result match for place: True / True
try transition motion
transition motion tried: True
4/50: Result match for pick: True / True
try transition motion
transition motion tried: True
4/50: Result match for place: True / True
try transition motion
transition motion tried: True
5/50: Result match for pick: True / True
try transition motion
transition motion tried: True
5/50: Result match for place: True / True
try transition motion
transition motion tried: True
6/50: Result match for pick: True / True
try transition motion
transition motion tried: False
6/50: Res

transition motion tried: True
49/50: Result match for place: True / True
try transition motion
transition motion tried: True
50/50: Result match for pick: True / True
try transition motion
transition motion tried: True
50/50: Result match for place: True / True


In [18]:
ACCURACY_CUT = 70
print("Accuracy cutline: {} %".format(ACCURACY_CUT))
for k, v in result_dict.items():
    v_arr = np.array(v)
    suc_idx = np.where(v_arr[:, 1])[0]
    fail_idx = np.where(np.logical_not(v_arr[:, 1]))[0]
    acc_val = np.round(np.mean(v_arr[:,0]==v_arr[:,1])*100, 1)
    acc_suc = np.round(np.mean(v_arr[suc_idx,0]==v_arr[suc_idx,1])*100, 1)
    acc_fail = np.round(np.mean(v_arr[fail_idx,0]==v_arr[fail_idx,1])*100, 1)
    print("="*60)
    print("{} accuracy: {} % / success accuracy: {} % / fail accuracy: {} % / success count:{}/{}".format(k, acc_val, acc_suc, acc_fail, len(suc_idx), len(v_arr)))
    assert acc_val>ACCURACY_CUT and acc_suc>ACCURACY_CUT, \
            "Accuracy {}/{} is lower than {}. Consider debugging the algorithm.".format(acc_val, acc_suc, ACCURACY_CUT)
    print("""OK with "{}" scenario""".format(k))

Accuracy cutline: 70 %
place accuracy: 94.9 % / success accuracy: 100.0 % / fail accuracy: 60.0 % / success count:34/39
OK with "place" scenario
pick accuracy: 82.0 % / success accuracy: 88.1 % / fail accuracy: 50.0 % / success count:42/50
OK with "pick" scenario


### planning: pick&place

In [20]:
from pkg.planning.constraint.constraint_common import calc_redundancy
from copy import deepcopy

Nmax_obj = 3
gtimer = GlobalTimer.instance()
gtimer.reset()
## add floor, ceiling
floor = Floor(gscene, "floor")
gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                   dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                   color=floor.geometry.color, display=True, collision=True, fixed=True)
# ceiling = Ceiling(gscene, "ceiling")

## set workplane
wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane)

gp_colliding = True
while gp_colliding:
    ## set goalplane
    gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
    gp_colliding = wp.is_overlapped_with(gp.geometry)
pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane)


## add object
obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

obj_set_list = []
for i_obj, obj in enumerate(obj_list):
    obj_pscene, handles = add_object(pscene, obj)
    obj_set_list.append((obj, obj_pscene, handles))
obj, obj_pscene, handles = obj_set_list[0]

obj_pscene.geometry.color = (0.8,0.2,0.2,1)
if VISUALIZE:
    gscene.set_rviz()

mplan.update_gscene()
initial_state = pscene.update_state(HOME_POSE)
pscene.set_object_state(initial_state)
from_state = initial_state.copy(pscene)

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

gtimer = GlobalTimer.instance()
gtimer.reset()

mplan.motion_filters = checkers_all
goal_nodes = [("gp",)+deepcopy(from_state.node)[1:]]
mplan.reset_log(True)
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=600, multiprocess=False, timeout=5)
elapsed = gtimer.toc("plan")/1000
schedules = ppline.tplan.find_schedules()
res = len(schedules)>0
if res:
    schedule = ppline.tplan.sort_schedule(schedules)[0]
    ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedule))
else:
    raise(RuntimeError("Failed to make plan - try again or debug"))

try: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0')
result: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
try: 0 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp')
result: 0 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
try: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0')
result: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
try: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')

try transition motion
transition motion tried: True
result: 2 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 2->8 (0.71/600.0 s, steps/err: 34(64.1069412231 ms)/0.00145340248647)
try: 8 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 8 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 3 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp')
result: 3 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
try: 7 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0')
result: 7 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
try: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0')
try transition motion
transition motion tried: True
result: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = success
branching: 0->9 (0.91/600.0 s, steps/err: 42(162.919998169 ms)/0.000974101880141)
try: 9 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp')
result: 9 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
try: 7 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp')
result: 7 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') =

result: 16 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 16->17 (1.61/600.0 s, steps/err: 41(69.8461532593 ms)/0.00163883783348)
try: 11 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp')
result: 11 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
try: 4 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0')
try transition motion
transition motion tried: True
result: 4 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = success
branching: 4->18 (1.7/600.0 s, steps/err: 44(86.1930847168 ms)/0.00113768729387)
try: 18 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp')
try transition motion
transition motion tried: True
result: 18 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 18->19 (1.75/600.0 s, steps/err: 15(52.4599552155 ms)/0.00170508292353)
try: 9 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp')
result: 9 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
try: 4 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 4 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 18 - ('wp

result: 6 - ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 6->29 (9.15/600.0 s, steps/err: 28(67.5449371338 ms)/0.00164882005111)
try: 29 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 29 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 18 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp')
result: 18 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
try: 25 - ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp')
result: 25 - ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp') = fail
try: 1 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp')
result: 1 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
try: 25 - ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp')
result: 25 - ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp') = fail
try: 16 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp')
result: 16 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
try: 21 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
try transition motion
transition motion tried: True
=============== try reaching goal from ('grip0', 'wp', 'wp') =======

result: 31 - ('grip0', 'wp', 'gp')->('wp', 'wp', 'gp') = success
branching: 31->40 (10.1/600.0 s, steps/err: 20(52.2661209106 ms)/0.0014735609556)
try: 40 - ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0')
result: 40 - ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0') = fail
try: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
try transition motion
transition motion tried: True
=============== try reaching goal from ('grip0', 'wp', 'wp') =================
result: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
branching: 0->41 (10.2/600.0 s, steps/err: 25(93.9531326294 ms)/0.00216378840243)
try: 41 - ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp')
result: 41 - ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
try: 11 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp')
result: 11 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
try: 40 - ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp')
result: 40 - ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
try: 23 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp')
result: 23 - ('wp

...

transition motion tried: True
result: 88 - ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0') = success
branching: 88->136 (40.53/600.0 s, steps/err: 107(843.706130981 ms)/0.00105213294313)
try: 136 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp')
result: 136 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
try: 76 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 76 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 136 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp')
result: 136 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
try: 41 - ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp')
result: 41 - ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
try: 101 - ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0')
result: 101 - ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0') = fail
try: 8 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 8 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 80 - ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0')
result: 80 - ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0') = fail
try: 18 - ('wp', 

transition motion tried: False
result: 36 - ('gp', 'wp', 'gp')->('gp', 'grip0', 'gp') = fail
try: 108 - ('grip0', 'gp', 'wp')->('wp', 'gp', 'wp')
try transition motion
transition motion tried: True
result: 108 - ('grip0', 'gp', 'wp')->('wp', 'gp', 'wp') = success
branching: 108->145 (51.51/600.0 s, steps/err: 10(54.5101165771 ms)/0.00158539960506)
try: 145 - ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp')
result: 145 - ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp') = fail
try: 70 - ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp')
result: 70 - ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
try: 38 - ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp')
result: 38 - ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
try: 130 - ('gp', 'grip0', 'gp')->('gp', 'gp', 'gp')
result: 130 - ('gp', 'grip0', 'gp')->('gp', 'gp', 'gp') = fail
try: 50 - ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0')
result: 50 - ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0') = fail
try: 143 - ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp')
result: 143 - ('wp', 'w

result: 104 - ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
try: 82 - ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp')
result: 82 - ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp') = fail
try: 154 - ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp')
try transition motion
transition motion tried: False
result: 154 - ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
try: 55 - ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp')
result: 55 - ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
try: 23 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp')
result: 23 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
try: 46 - ('gp', 'grip0', 'gp')->('gp', 'gp', 'gp')
result: 46 - ('gp', 'grip0', 'gp')->('gp', 'gp', 'gp') = fail
try: 106 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
result: 106 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 55 - ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp')
result: 55 - ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
try: 111 - ('gp', 'grip0', 'gp')->('gp', 'gp', 'gp')
result: 111 - ('gp', 'grip0',

result: 9 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
try: 124 - ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp')
result: 124 - ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
try: 159 - ('gp', 'wp', 'gp')->('grip0', 'wp', 'gp')
result: 159 - ('gp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
try: 74 - ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0')
result: 74 - ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = fail
try: 58 - ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp')
result: 58 - ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
try: 39 - ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0')
result: 39 - ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0') = fail
try: 52 - ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp')
result: 52 - ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp') = fail
try: 133 - ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp')
result: 133 - ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp') = fail
try: 69 - ('gp', 'grip0', 'gp')->('gp', 'gp', 'gp')
result: 69 - ('gp', 'grip0', 'gp')->('gp', 'gp', 'gp') = fail
try: 134 - ('wp', 'wp

try transition motion
transition motion tried: True
result: 75 - ('wp', 'grip0', 'gp')->('wp', 'wp', 'gp') = success
branching: 75->172 (64.44/600.0 s, steps/err: 10(54.8429489136 ms)/0.00190399933982)
try: 172 - ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0')
try transition motion
transition motion tried: True
result: 172 - ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0') = success
branching: 172->173 (64.66/600.0 s, steps/err: 71(212.759017944 ms)/0.00110550568078)
try: 127 - ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp')
result: 127 - ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
try: 57 - ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0')
result: 57 - ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0') = fail
try: 41 - ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp')
result: 41 - ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
try: 39 - ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp')
result: 39 - ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
try: 73 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp')
result: 73 - ('wp', 'wp', 'grip

In [21]:
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedule))